<a href="https://colab.research.google.com/github/tanvircr7/RAG-langchain-openai/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [5]:
# GET API KEY
from google.colab import userdata
APIKEY = userdata.get('OPENAI_API_KEY')

In [2]:
!pip install langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.6/389.6 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 30.6 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.52.2
    Uninstalling openai-1.52.2:
      Successfully uninstalled openai-1.52.2


In [8]:
from langchain_openai import ChatOpenAI

In [9]:
llm = ChatOpenAI(model = "gpt-4o",
                 temperature = 0,
                 api_key = APIKEY)

In [11]:
prompt = f"""
provide python code snippet using the yfinance library to download stock data.
the tickers are {assets} from {start_year} to {end_year}.
"""

In [12]:
response = llm(prompt)

<ipython-input-12-a8a9d1e67c96>:1: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm(prompt)


In [15]:
# response
# response.content

"To download stock data using the `yfinance` library for the specified tickers from 2022 to 2023, you can use the following Python code snippet. Note that the tickers should be in the correct format for Yahoo Finance, and I've adjusted them accordingly:\n\n```python\nimport yfinance as yf\n\n# Define the tickers\ntickers = ['AAPL', 'AMZN', 'BTC-USD', 'GOOGL', 'META', 'MSFT', 'NVDA', 'SPY', 'TSLA']\n\n# Define the start and end dates\nstart_date = '2022-01-01'\nend_date = '2023-12-31'\n\n# Download the data\ndata = yf.download(tickers, start=start_date, end=end_date)\n\n# Display the data\nprint(data)\n```\n\nThis code snippet will download the historical stock data for the specified tickers from January 1, 2022, to December 31, 2023. The data will include various metrics such as Open, High, Low, Close, Adj Close, and Volume for each ticker."

In [13]:
from IPython.display import display, Markdown
display(Markdown(response.content))

To download stock data using the `yfinance` library for the specified tickers from 2022 to 2023, you can use the following Python code snippet. Note that the tickers should be in the correct format for Yahoo Finance, and I've adjusted them accordingly:

```python
import yfinance as yf

# Define the tickers
tickers = ['AAPL', 'AMZN', 'BTC-USD', 'GOOGL', 'META', 'MSFT', 'NVDA', 'SPY', 'TSLA']

# Define the start and end dates
start_date = '2022-01-01'
end_date = '2023-12-31'

# Download the data
data = yf.download(tickers, start=start_date, end=end_date)

# Display the data
print(data)
```

This code snippet will download the historical stock data for the specified tickers from January 1, 2022, to December 31, 2023. The data will include various metrics such as Open, High, Low, Close, Adj Close, and Volume for each ticker.

In [20]:
def output(prompt):
  response = llm(prompt)
  display(Markdown(response.content))

-------------------------------

In [23]:
pip install -q pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 4.4 MB/s eta 0:00:00


In [24]:
pip install -q docx2txt

  Preparing metadata (setup.py) ... done


In [25]:
pip install -q wikipedia

  Preparing metadata (setup.py) ... done


In [26]:
pip install --upgrade -q openai langchain tiktoken

Loading Documents

In [52]:
# loading PDF, DOCX and TXT files as LangChain Documents
def load_document(file):
    import os
    name, extension = os.path.splitext(file)

    if extension == '.pdf':
        from langchain.document_loaders import PyPDFLoader
        print(f'Loading {file}')
        loader = PyPDFLoader(file)
    elif extension == '.docx':
        from langchain.document_loaders import Docx2txtLoader
        print(f'Loading {file}')
        loader = Docx2txtLoader(file)
    elif extension == '.txt':
        from langchain.document_loaders import TextLoader
        loader = TextLoader(file)
    else:
        print('Document format is not supported!')
        return None

    data = loader.load()
    return data


# wikipedia
def load_from_wikipedia(query, lang='en', load_max_docs=2):
    from langchain.document_loaders import WikipediaLoader
    loader = WikipediaLoader(query=query, lang=lang, load_max_docs=load_max_docs)
    data = loader.load()
    return data


In [53]:
# chunk data

In [54]:
def chunk_data(data, chunk_size=256):
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=0)
    chunks = text_splitter.split_documents(data)
    return chunks


In [55]:
def print_embedding_cost(texts):
    import tiktoken
    enc = tiktoken.encoding_for_model('text-embedding-3-small')
    total_tokens = sum([len(enc.encode(page.page_content)) for page in texts])
    print(f'Total Tokens: {total_tokens}')
    print(f'Embedding Cost in USD: {total_tokens / 1000 * 0.00002:.6f}')

In [56]:
def insert_or_fetch_embeddings(index_name, chunks):
    # importing the necessary libraries and initializing the Pinecone client
    import pinecone
    from langchain_community.vectorstores import Pinecone
    from langchain_openai import OpenAIEmbeddings
    from pinecone import PodSpec


    pc = pinecone.Pinecone()

    embeddings = OpenAIEmbeddings(model='text-embedding-3-small', dimensions=1536)  # 512 works as well

    # loading from existing index
    if index_name in pc.list_indexes():
        print(f'Index {index_name} already exists. Loading embeddings ... ', end='')
        vector_store = Pinecone.from_existing_index(index_name, embeddings)
        print('Ok')
    else:
        # creating the index and embedding the chunks into the index
        print(f'Creating index {index_name} and embeddings ...', end='')

        # creating a new index
        pc.create_index(
            name=index_name,
            dimension=1536,
            metric='cosine',
            spec=PodSpec(
                environment='gcp-starter'
            )
        )

        # processing the input documents, generating embeddings using the provided `OpenAIEmbeddings` instance,
        # inserting the embeddings into the index and returning a new Pinecone vector store object.
        vector_store = Pinecone.from_documents(chunks, embeddings, index_name=index_name)
        print('Ok')

    return vector_store

In [57]:
def delete_pinecone_index(index_name='all'):
    import pinecone
    pc = pinecone.Pinecone()

    if index_name == 'all':
        indexes = pc.list_indexes().names()
        print('Deleting all indexes ... ')
        for index in indexes:
            pc.delete_index(index)
        print('Ok')
    else:
        print(f'Deleting index {index_name} ...', end='')
        pc.delete_index(index_name)
        print('Ok')


# **QA**

In [66]:
def ask_and_get_answer(vector_store, q, k=3):
    from langchain.chains import RetrievalQA
    from langchain_openai import ChatOpenAI

    llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=1, openai_api_key=APIKEY)

    retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k': k})

    chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

    answer = chain.invoke(q)
    return answer


In [59]:
pip install -q chromadb

In [60]:
def create_embeddings_chroma(chunks, persist_directory='./chroma_db'):
    from langchain.vectorstores import Chroma
    from langchain_openai import OpenAIEmbeddings

    # Instantiate an embedding model from OpenAI (smaller version for efficiency)
    embeddings = OpenAIEmbeddings(model='text-embedding-3-small', dimensions=1536, openai_api_key=APIKEY)

    # Create a Chroma vector store using the provided text chunks and embedding model,
    # configuring it to save data to the specified directory
    vector_store = Chroma.from_documents(chunks, embeddings, persist_directory=persist_directory)

    return vector_store  # Return the created vector store


In [70]:
def load_embeddings_chroma(persist_directory='./chroma_db'):
    from langchain.vectorstores import Chroma
    from langchain_openai import OpenAIEmbeddings

    # Instantiate the same embedding model used during creation
    embeddings = OpenAIEmbeddings(model='text-embedding-3-small', dimensions=1536, openai_api_key = APIKEY)

    # Load a Chroma vector store from the specified directory, using the provided embedding function
    vector_store = Chroma(persist_directory=persist_directory, embedding_function=embeddings)

    return vector_store  # Return the loaded vector store


In [62]:
!pip install langchain_community

In [63]:
from google.colab import files

# This will prompt you to select files to upload
uploaded = files.upload()

# Display the names of the uploaded files
for filename in uploaded.keys():
    print(f'Uploaded file "{filename}" with size {len(uploaded[filename])} bytes')

Saving us_constitution.pdf to us_constitution (2).pdf
Uploaded file "us_constitution (2).pdf" with size 170876 bytes


In [64]:
# Loading the pdf document into LangChain
data = load_document('us_constitution.pdf')

# Splitting the document into chunks
chunks = chunk_data(data, chunk_size=256)

# Creating a Chroma vector store using the provided text chunks and embedding model (default is text-embedding-3-small)
vector_store = create_embeddings_chroma(chunks)

Loading us_constitution.pdf


In [67]:
# Asking questions
q = 'What are the ammendments?'
answer = ask_and_get_answer(vector_store, q)
print(answer)

{'query': 'What are the ammendments?', 'result': 'The amendments referenced in the context provided are the 23rd, 24th, and Second Amendments to the United States Constitution.'}


In [68]:
print(answer['result'])

The amendments referenced in the context provided are the 23rd, 24th, and Second Amendments to the United States Constitution.


In [71]:
# Load a Chroma vector store from the specified directory (default ./chroma_db)
db = load_embeddings_chroma()
q = 'Explain the second ammendment?'
answer = ask_and_get_answer(vector_store, q)
print(answer)

<ipython-input-70-1632b5ad7438>:9: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_store = Chroma(persist_directory=persist_directory, embedding_function=embeddings)


{'query': 'Explain the second ammendment?', 'result': 'The Second Amendment of the United States Constitution protects the right of the people to keep and bear arms. It is often interpreted as ensuring that individuals have the right to own and carry firearms. The Second Amendment states: "A well regulated Militia, being necessary to the security of a free State, the right of the people to keep and bear Arms, shall not be infringed."'}


In [72]:
# We can't ask follow-up questions. There is no memory (chat history) available.
q = 'Who made the 2nd ammendment?'
answer = ask_and_get_answer(vector_store, q)
print(answer['result'])

The Second Amendment to the United States Constitution was made by the Constitutional Convention.
